In [1]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from network_structure.reservoir import reservoir
from network_structure.spatial_reservoir import spatial_network
import path, os
from time import time

### Initialize the Reservoir

In [141]:
size_inp = "1_1_3"
size_res = "40_5_5"
size_out = "1_1_3"

r = spatial_network(3)
r.set_size_of("inp", [int(ele) for ele in size_inp.split("_")])
r.generate_spatial_structure_of("inp", container_size=[1,1,6], distribution="uniform")
r.embedding_of("inp", axis=[])
r.set_position_and_rotation_of("inp", position=[-20,0,0])

r.set_size_of("res", [int(ele) for ele in size_res.split("_")])
r.generate_spatial_structure_of("res", container_size=[40,5,5])
r.embedding_of("res", axis=[])
r.set_position_and_rotation_of("res", position=[0,0,0])

r.set_size_of("out", [int(ele) for ele in size_out.split("_")])
r.generate_spatial_structure_of("out", container_size=[1,1,6], distribution="uniform")
r.embedding_of("out", axis=[])
r.set_position_and_rotation_of("out", position=[20,0,0])
r.initial_weight()

### Define Distance Function for Generating the Weight

In [142]:
def distance(x,y):
    return np.exp(-0.05*(x-y)**2)
#attention to the sum of axiss!!(here I use multiply operation)
print(distance(1,2)**3, distance(1,3)**3, distance(1,4)**3)
def distance_inv(x,y):
    return np.exp(-1*5/(x-y)**2)
#attention to the sum of axiss!!(here I use multiply operation)
print(distance_inv(1,2)**3, distance_inv(1,3)**3, distance_inv(1,4)**3)

0.8607079764250578 0.5488116360940264 0.25924026064589156
3.059023205018258e-07 0.023517745856009107 0.18887560283756177


### Connect The Reservoir

In [143]:
r.add_sign(0.5)
r.connect_with_coordinates("inp2res", distance_function=distance)
r.connect_with_coordinates("res2res", distance_function=distance, scaling=1)
#r.connect_with_coordinates("out2res", distance_function=distance)
function_type = "sigmoid"
#function_type = "no_activation"
r.set_activation_function(function_type)
r.set_leaking_rate()
print(time()-a)
a = time()
#r.cut_weight_with_threshold("inp2res", threshold_lower=0.001)
r.cut_weight_with_threshold("res2res", threshold_lower=0.000001, threshold_upper=1.1)
r.show_reservoir_status()
print(time()-a)

1540.9299244880676
size of the reservoir: 3 1000 3
activation function type: sigmoid
leaking_rate: 0.2
the range of the weight inp2res is [+0.00000, +0.00000] and [+0.00000, +1.00000], sparsity:1.00000
the range of the weight res2res is [-1.00000, -0.00000] and [+0.00000, +1.00000], sparsity:0.63744
the range of the weight res2out is [+0.00000, +0.00000] and [+0.00000, +0.00000], sparsity:1.00000
the range of the weight out2res is [+0.00000, +0.00000] and [+0.00000, +0.00000], sparsity:1.00000
the current spectral radius is 21.880298
0.3118300437927246


### Prune

In [144]:
#r.cut_weight_with_threshold("inp2res", threshold_lower=0.001)
r.cut_weight_with_threshold("res2res", threshold_lower=0.7, threshold_upper=1)
r.show_reservoir_status()

size of the reservoir: 3 1000 3
activation function type: sigmoid
leaking_rate: 0.2
the range of the weight inp2res is [+0.00000, +0.00000] and [+0.00000, +1.00000], sparsity:1.00000
the range of the weight res2res is [-1.00000, -0.70002] and [+0.70002, +1.00000], sparsity:0.05069
the range of the weight res2out is [+0.00000, +0.00000] and [+0.00000, +0.00000], sparsity:1.00000
the range of the weight out2res is [+0.00000, +0.00000] and [+0.00000, +0.00000], sparsity:1.00000
the current spectral radius is 15.295809


### Adjust Spectral Radius

In [ ]:
sr_of_matrix = []
max_sr_of_submatrices = []
for i in range(5):
    r.show_reservoir_status(update_without_printout=True)
    sr_of_matrix.append(r.spectral_radius)
    r.adjust_all_spectral_radius_to_value(nodes_type="res", sr=1, loops=1)
    srl = []
    for index_node in range(r.num["res"]):
        _, sr = r.get_connection_situation_of("res", index_node)
        srl.append(sr)
    srl = np.asarray(srl)
    max_sr_of_submatrices.append(np.max(srl))
    print(sr_of_matrix[i], max_sr_of_submatrices[i])

### Generate Toy Input

In [ ]:
inp = np.zeros([1, 1200])
for i in range(5):
    index = np.random.randint(800)
    for j in range(30):
        inp[0, index+j] = 1
#        inp[1, index+j] = -1
xx = np.linspace(0,1000,1200)
plt.plot(xx, inp[0])
plt.show()

### Plot Reservoir Evolution

In [ ]:
plt.close()
r.set_activation_function("sigmoid")
r.clear_value()
a = time() 
scale = np.asarray(r.spectral_radius_list)
#fig = plt.figure(figsize=[12,8])
#ax = fig.add_subplot(111, projection='3d')
for i in range(1000):
    fig = plt.figure(figsize=[15,8])
    ax = fig.add_subplot(111, projection='3d')
    r.run(inp.T[i])
    r.plot_coordinates(ax, "inp", nodes_size=100, nodes_color=r.value["inp"][:,0], vmin=-0.1, vmax=0.1)
    r.plot_coordinates(ax, "res", nodes_size=10, nodes_color=r.update["res"][:,0], vmin=-0.00002, vmax=0.00002)
    print(i, np.min(r.update["res"]), np.max(r.update["res"]))
    plt.savefig("iamge_structure/anime_"+str(i))
    plt.close()
plt.savefig("spatial_reservoir")
print(time()-a)

### Plot Reservoir with Weight

In [ ]:
a = time()
fig = plt.figure(figsize=[12,8])
ax = fig.add_subplot(111, projection='3d')
weight_list = ["res2res"]
for weight in weight_list:
    r.plot_weight(ax, weight, linewidth=0.2)
plt.savefig("spatial_reservoir_15", dpi=800)
plt.show()
print(time()-a)

### Save Reservoir

In [ ]:
function_type = "sigmoid"
#function_type = "no_activation"
#function_type = "relu"
r.set_activation_function(function_type)
task_name = "three_bit_flip_flop"
#task_name = "narma"
global_parameter = "size_inp" + size_inp + "_res" + size_res + "_out" + size_out
parameter = "_sr134_" + "sparsity0049_" + function_type
file_name = path.spatial_network + "/" + task_name + "/" + global_parameter + parameter
print(file_name)
#os.mkdir(path.network)
#os.mkdir(path.spatial_network)
#os.mkdir(path.spatial_network + "/" + task_name)
np.save(file_name, r)

### Plot Weight

In [ ]:
c = r.sort_weight()
start = 0
end = 100
weight_local = r.weight["res2res"][start:end, start:end]
linspace = np.linspace(1, weight_local.shape[0], weight_local.shape[0])
grid = np.asarray([[i,j] for i in linspace for j in linspace])

plt.figure(figsize=[12,8])
from matplotlib import cm
plt.scatter(grid[:,0], grid[:,1], 
            c=weight_local.reshape(-1), cmap="RdYlGn", vmin=-2, vmax=2, s=10, marker="s")
plt.show()